# Compiling Consumption Data

In [1]:
import pandas as pd

In [2]:
# Load all consumptio data before intervention (This runs for ages)

df1 = pd.read_excel('A02A-Transcript_Status_(short_report)-File1.xlsx', header=14)
df2 = pd.read_excel('A02A-Transcript_Status_(short_report)-File2.xlsx', header=14)
df3 = pd.read_excel('A02A-Transcript_Status_(short_report)-File3.xlsx', header=14)
df4 = pd.read_excel('A02A-Transcript_Status_(short_report)-File4.xlsx', header=14)
df5 = pd.read_excel('A02A-Transcript_Status_(short_report)-File5.xlsx', header=14)

In [3]:
#load data after intervention
consumption_new_metadata = pd.read_excel('Consumption_Data_After.xlsx', header=14)

In [4]:
#concat the datasets
all_consumption = pd.concat([df1, df2, df3, df4, df5, consumption_new_metadata], axis=0)

In [5]:
#get the date
def choose_date(row):
    if 'completed' in row['Transcript Status'].lower():
       return row['Transcript Completed Date']
    else:
      return row['Transcript Assigned Date']

all_consumption['date'] = all_consumption.apply(choose_date, axis=1).dt.date

In [6]:
consumption_per_LO = all_consumption.pivot_table(index=['Training Object ID', 'date'], columns='Transcript Status', values='User ID', aggfunc='count').reset_index().fillna(0)

In [7]:
# drop duplicates
type_and_providers = all_consumption[['Training Object ID', 'Training Type', 'Division']].drop_duplicates()

In [8]:
#get consumption per learning object
consumption_per_LO['Registrations'] = consumption_per_LO['Registered']
consumption_per_LO['Completions'] = consumption_per_LO['Completed'] + consumption_per_LO['Completed (Equivalent)']
consumption_df = consumption_per_LO[['Training Object ID', 'date', 'Registrations', 'Completions']]

In [9]:
# consumption_df.to_csv('consumption_df.csv')

In [10]:
consumption_df['date']= pd.to_datetime(consumption_df.date)
consumption_df['date'].dtype

/tmp/ipykernel_98/417411795.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consumption_df['date']= pd.to_datetime(consumption_df.date)


dtype('<M8[ns]')

# labelled data

Read data

In [11]:
# import libraries and data for data engineering
import pandas as pd
import itertools
consumption_df = pd.read_csv('consumption_df.csv', index_col='Unnamed: 0').drop(columns='Completions')
labelled_data = pd.read_csv('treatment_control.csv')[['NVS_Training_Object_ID', 'initial_desc_length', 'filled_in']].rename(columns={'filled_in':'in_test_group'})
LO_meta_raw = pd.read_csv('clean_subset.csv')[['NVS_Training_Object_ID', 'NVS_Course_Training_Type', 'NVS_Course_Training_Provider']]

Manipulate data

In [12]:
# get initial LO popularity
popularity_df = consumption_df[
    consumption_df['date']<='2023-05-15'
    ].groupby('Training Object ID').sum().reset_index().rename(columns={'Registrations':'popularity'})

# one dataframe with all metadata
LO_metadata = labelled_data.merge(LO_meta_raw)

LO_metadata = LO_metadata.merge(
    popularity_df, 
    left_on='NVS_Training_Object_ID', 
    right_on='Training Object ID', 
    how='left').drop(columns=['Training Object ID'])
    
LO_metadata['popularity'] = LO_metadata['popularity'].fillna(0)

LO_metadata['popularity_Bins'] = pd.cut(
    LO_metadata['popularity'], 
    bins=[0, 1, 10, 50, 5000], 
    include_lowest=True,
    right=False, 
    labels = ['0', '1-10', '11-50', '50+'])

In [13]:
# We need data for every LO, for every period in our data. Here we first create a placeholder, then populate it w/ data

# create placeholder
ref_dataframe = pd.DataFrame(
    data=list(itertools.product(
        pd.date_range(
            pd.to_datetime('2022-04-01'),
            pd.to_datetime('2023-05-31')
            ),
        labelled_data['NVS_Training_Object_ID'].unique())),
    columns = ['date', 'NVS_Training_Object_ID'])

# ensure data types
ref_dataframe['date'] = ref_dataframe['date'].astype(str)
consumption_df['date'] = consumption_df['date'].astype(str)

# and consumption data into placeholder
ref_dataframe = ref_dataframe.merge(
    consumption_df, 
    how='left', 
    left_on=['NVS_Training_Object_ID', 'date'], 
    right_on=['Training Object ID', 'date']
).drop(columns='Training Object ID')

# where we didn't match, fill nas with zero
ref_dataframe['Registrations'] = ref_dataframe['Registrations'].fillna(0)

# set datetime index for resampling
ref_dataframe = ref_dataframe.set_index('date')
ref_dataframe.index = pd.to_datetime(ref_dataframe.index)

# resample weekly
ref_weekly_df = ref_dataframe.groupby('NVS_Training_Object_ID').resample('W-WED').sum().reset_index()
ref_weekly_df.head()

# finally, merge final (weekly) consumption with desired metadata
regression_df = ref_weekly_df.merge(LO_metadata)

# get dummy for intervention, i need it for the time variable for the difference in difference regression
regression_df['after_intervention'] = 0
regression_df.loc[regression_df['date'] > pd.to_datetime('2023-05-17'), 'after_intervention'] = 1

In [14]:
# final df:
regression_df

,NVS_Training_Object_ID,date,Registrations,initial_desc_length,in_test_group,NVS_Course_Training_Type,NVS_Course_Training_Provider,popularity,popularity_Bins,after_intervention
0,000ed53c-8372-493f-87fa-06858911ef6d,2022-04-06,0.0,140,1,Material,NIBR,131.0,50+,0
1,000ed53c-8372-493f-87fa-06858911ef6d,2022-04-13,0.0,140,1,Material,NIBR,131.0,50+,0
2,000ed53c-8372-493f-87fa-06858911ef6d,2022-04-20,0.0,140,1,Material,NIBR,131.0,50+,0
3,000ed53c-8372-493f-87fa-06858911ef6d,2022-04-27,0.0,140,1,Material,NIBR,131.0,50+,0
4,000ed53c-8372-493f-87fa-06858911ef6d,2022-05-04,0.0,140,1,Material,NIBR,131.0,50+,0
...,...,...,...,...,...,...,...,...,...,...
271872,fffdd332-cfc8-4804-95b3-a777c7d49fa0,2023-05-03,0.0,19,0,Material,NBS,0.0,0,0
271873,fffdd332-cfc8-4804-95b3-a777c7d49fa0,2023-05-10,0.0,19,0,Material,NBS,0.0,0,0
271874,fffdd332-cfc8-4804-95b3-a777c7d49fa0,2023-05-17,0.0,19,0,Material,NBS,0.0,0,0
271875,fffdd332-cfc8-4804-95b3-a777c7d49fa0,2023-05-24,0.0,19,0,Material,NBS,0.0,0,1


In [15]:
# Create dummy variables
dummy_df = pd.get_dummies(regression_df['popularity_Bins'], prefix='popularity_Bins')

# Concatenate the dummy variables with the original DataFrame
new_regression_df = pd.concat([regression_df, dummy_df], axis=1)

In [16]:
new_regression_df

,NVS_Training_Object_ID,date,Registrations,initial_desc_length,in_test_group,NVS_Course_Training_Type,NVS_Course_Training_Provider,popularity,popularity_Bins,after_intervention,popularity_Bins_0,popularity_Bins_1-10,popularity_Bins_11-50,popularity_Bins_50+
0,000ed53c-8372-493f-87fa-06858911ef6d,2022-04-06,0.0,140,1,Material,NIBR,131.0,50+,0,0,0,0,1
1,000ed53c-8372-493f-87fa-06858911ef6d,2022-04-13,0.0,140,1,Material,NIBR,131.0,50+,0,0,0,0,1
2,000ed53c-8372-493f-87fa-06858911ef6d,2022-04-20,0.0,140,1,Material,NIBR,131.0,50+,0,0,0,0,1
3,000ed53c-8372-493f-87fa-06858911ef6d,2022-04-27,0.0,140,1,Material,NIBR,131.0,50+,0,0,0,0,1
4,000ed53c-8372-493f-87fa-06858911ef6d,2022-05-04,0.0,140,1,Material,NIBR,131.0,50+,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271872,fffdd332-cfc8-4804-95b3-a777c7d49fa0,2023-05-03,0.0,19,0,Material,NBS,0.0,0,0,1,0,0,0
271873,fffdd332-cfc8-4804-95b3-a777c7d49fa0,2023-05-10,0.0,19,0,Material,NBS,0.0,0,0,1,0,0,0
271874,fffdd332-cfc8-4804-95b3-a777c7d49fa0,2023-05-17,0.0,19,0,Material,NBS,0.0,0,0,1,0,0,0
271875,fffdd332-cfc8-4804-95b3-a777c7d49fa0,2023-05-24,0.0,19,0,Material,NBS,0.0,0,1,1,0,0,0


In [17]:
new_regression_df = new_regression_df.rename(columns={'popularity_Bins_1-10': 'popularity_Bins_1_10'})
new_regression_df = new_regression_df.rename(columns={'popularity_Bins_11-50': 'popularity_Bins_11_50'})
new_regression_df = new_regression_df.rename(columns={'popularity_Bins_50+': 'popularity_Bins_more_than_50'})

In [18]:
# Create dummy variables
dummy_df1 = pd.get_dummies(new_regression_df['NVS_Course_Training_Type'], prefix='NVS_Course_Training_Type')

# Concatenate the dummy variables with the original DataFrame
new_regression_df = pd.concat([new_regression_df, dummy_df1], axis=1)

In [19]:
new_regression_df.columns

Index(['NVS_Training_Object_ID', 'date', 'Registrations',
       'initial_desc_length', 'in_test_group', 'NVS_Course_Training_Type',
       'NVS_Course_Training_Provider', 'popularity', 'popularity_Bins',
       'after_intervention', 'popularity_Bins_0', 'popularity_Bins_1_10',
       'popularity_Bins_11_50', 'popularity_Bins_more_than_50',
       'NVS_Course_Training_Type_Curriculum', 'NVS_Course_Training_Type_Event',
       'NVS_Course_Training_Type_Material',
       'NVS_Course_Training_Type_Online Class',
       'NVS_Course_Training_Type_Session', 'NVS_Course_Training_Type_Test',
       'NVS_Course_Training_Type_Video'],
      dtype='object')

In [20]:
new_regression_df.to_csv('new_regression_df.csv')

Control Variables:

- "Popularity Bins" = Previous Consumption Categories (creating dummies categorical control variable, total registrations from April 2022 - April 2023, four categories based on the number of registrations: 0, 1-10, 11-50, 50+)

- "Initial Description length" = Description length before intervention (numeric control variable, number of characters in NVS_Course_Description)


- "Training Type" = Training type (categorical control variable, e.g., Video, Material, Online Course, etc.)

In [21]:
import statsmodels.api as sm
from tabulate import tabulate

In [22]:
df_for_sm = new_regression_df.copy()

In [27]:
import statsmodels.api as sm


# Combine the variables into a single DataFrame
df_for_sm['interaction'] = df_for_sm['in_test_group'] * df_for_sm['after_intervention']

# Define the regression formula
formula = 'Registrations ~ in_test_group + after_intervention + interaction + initial_desc_length + popularity_Bins_1_10 + popularity_Bins_11_50 + popularity_Bins_more_than_50 + NVS_Course_Training_Type_Material + NVS_Course_Training_Type_Video + NVS_Course_Training_Type_Session + NVS_Course_Training_Type_Test'
model = sm.formula.ols(formula=formula, data=df_for_sm)
#fit with robust standart errors
results = model.fit(cov_type='HC3') 
regression_table = results.summary()
print(regression_table)

                            OLS Regression Results                            
Dep. Variable:          Registrations   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     112.2
Date:                Sun, 11 Jun 2023   Prob (F-statistic):          2.44e-257
Time:                        09:23:23   Log-Likelihood:            -1.1046e+06
No. Observations:              271877   AIC:                         2.209e+06
Df Residuals:                  271865   BIC:                         2.209e+06
Df Model:                          11                                         
Covariance Type:                  HC3                                         
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Interc

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=09374e14-43a4-4eee-b362-608228cad85e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>